# These are the humble beginnings for pyTorch
The following [tutorial](https://pytorch.org/tutorials/beginner/basics/intro.html) has been used!



## Working with data

In [7]:
# This is just the great import of all the libraries
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets 
from torchvision.transforms import ToTensor


# Then we download training data, which my computer is probably gonna find highly
# irresponsible
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)

# And also downloading the test data
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
)

batch_size = 64

# Then we create something called data loaders:
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
    print(f"shape of X [N, C, H, W]: {X.shape}")
    print(f"shape of y: {y.shape} {y.dtype}")
    break
    

shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64


## Creating Models
So when we create a neural network in pyTorch it is apparently just creating a class which inherits from
the nn.Module class.
If classes are a little unfamiliar check out this great [link](https://www.w3schools.com/python/python_classes.asp). If inheritence is also a little unfamiliar,
this [link](https://www.w3schools.com/python/python_inheritance.asp) does wonders!
So here is an example of defining a neural network:

In [14]:
# I am uncertain how this exactly works, and it should probably be homework!
# Get CPU or GPU for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters

When we train a model we obviously need a loss function and an optimizer.

Which is what we will be looking at here!

In [15]:
# lr stands for learning rate
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


# In one instance of a training loop our network (the model) makes predictions
# based on the trainning dataset (fed to the model in batches),
# then it backpropagates the prediction error to adjust the model parameters (those we optimise)
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# To ensure that the model is actually learning something, we test its performance
# against the test dataset.
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


# Finally, the training process is conducted over several epochs (runs through the training set)
# We print the models loss and accuracy at each epoch, to show whether the procedure
# actually advances
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.299400  [    0/60000]
loss: 2.286723  [ 6400/60000]
loss: 2.269109  [12800/60000]
loss: 2.275484  [19200/60000]
loss: 2.245312  [25600/60000]
loss: 2.228361  [32000/60000]
loss: 2.242835  [38400/60000]
loss: 2.204467  [44800/60000]
loss: 2.208388  [51200/60000]
loss: 2.187904  [57600/60000]
Test Error: 
 Accuracy: 43.8%, Avg loss: 2.170998 

Epoch 2
-------------------------------
loss: 2.173972  [    0/60000]
loss: 2.166065  [ 6400/60000]
loss: 2.114031  [12800/60000]
loss: 2.139895  [19200/60000]
loss: 2.086643  [25600/60000]
loss: 2.033622  [32000/60000]
loss: 2.072351  [38400/60000]
loss: 1.990450  [44800/60000]
loss: 2.007109  [51200/60000]
loss: 1.942393  [57600/60000]
Test Error: 
 Accuracy: 55.2%, Avg loss: 1.929757 

Epoch 3
-------------------------------
loss: 1.955655  [    0/60000]
loss: 1.924351  [ 6400/60000]
loss: 1.816357  [12800/60000]
loss: 1.861954  [19200/60000]
loss: 1.757305  [25600/60000]
loss: 1.704592  [32000/600

## Saving models

Something about saving the model (very informative... I know)


In [16]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Loading models

Saving and loading models seems like two sides of the same coin... right?

In [17]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

# The loaded model can then be used to make predictions:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: '{predicted}', Actual: '{actual}'")
    

Predicted: 'Ankle boot', Actual: 'Ankle boot'
